# Code-to-Code Comparison: IEA Task 26

National Renewable Energy Laboratory\
Rob Hammond\
10 January 2023

**Note**: The results in this notebook will differ from those in prior versions quite significantly in some places due to the numerous logic changes that have both updated and improved the features provided by WOMBAT, and fixed previously unidentified bugs.

In [1]:
from copy import deepcopy
from time import perf_counter
from pprint import pprint

import yaml
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core.library import IEA_26, load_yaml

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

In [2]:
tech_salary_annual = 100000
techs = 30
capacity = 400 * 1000  # 400MW -> kW
tech_salary_annual * techs / capacity

7.5

In [3]:
configs = [
    "requests",
    "requests_100pct_reduction",
    "one_mobilization",
    "one_mobilization_100pct_reduction",
    "two_mobilizations",
    "two_mobilizations_100pct_reduction",
    "three_mobilizations",
    "three_mobilizations_100pct_reduction",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "total annual costs": [],
    "technicians": [],
    "materials": [],
    "vessels": [],
    "ctv cost": [],
    "hlv cost": [],
    "dsv cost": [],
    "cab cost": [],
    "manual reset": [],
    "minor repair": [],
    "major repair": [],
    "major replacement": [],
    "remote reset": [],
    "annual service": [],
    "bos": [],  #  substructure inspection + scour repair + substation inspection + small/large transformer repairs
    "total downtime": [],
    "ctv utilization": [],
    "hlv utilization": [],
    "dsv utilization": [],
    "cab utilization": [],
}
metrics_dict = {}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(IEA_26, f"{config}.yaml")
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(45)} | {(end - start) / 60:.2f} m")
    metrics_dict[config] = sim.metrics

                              iea_26_requests | 1.48 m
             iea_26_requests_100pct_reduction | 1.55 m
                      iea_26_one_mobilization | 1.67 m
     iea_26_one_mobilization_100pct_reduction | 1.81 m
                     iea_26_two_mobilizations | 1.66 m
    iea_26_two_mobilizations_100pct_reduction | 1.65 m
                   iea_26_three_mobilizations | 1.60 m
  iea_26_three_mobilizations_100pct_reduction | 1.61 m


In [5]:
for config, metrics in metrics_dict.items():
    
    # Gather the results of interest
    years = metrics.events.year.unique().shape[0]
    mil = 1000000
    
    availability = metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
    availability_production = metrics.production_based_availability(frequency="project", by="windfarm").values[0][0]
    cf_net = metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
    cf_gross = metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
    power_production = metrics.power_production(frequency="project", by="windfarm").values[0][0]
    completion_rate = metrics.task_completion_rate(which="both", frequency="project").values[0][0]
    parts = metrics.events[["materials_cost"]].sum().sum()
    techs = metrics.project_fixed_costs(frequency="project", resolution="medium").labor[0]
    total = metrics.events[["total_cost"]].sum().sum()

    equipment = metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Jack-up Vessel" in el]].sum().sum()
    dsv = equipment[[el for el in equipment.columns if "Diving Support Vessel" in el]].sum().sum()
    cab = equipment[[el for el in equipment.columns if "Cable Laying Vessel" in el]].sum().sum()

    times = metrics.process_times()
    times = times / years / 24 / 100  # events per turbine and year
    
    utilization = metrics.service_equipment_utilization(frequency="project")
    ctv_ur = utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]].mean().mean()
    hlv_ur = utilization[[el for el in utilization.columns if "Jack-up Vessel" in el]].mean().mean()
    dsv_ur = utilization[[el for el in utilization.columns if "Diving Support Vessel" in el]].mean().mean()
    cab_ur = utilization[[el for el in utilization.columns if "Cable Laying Vessel" in el]].mean().mean()

    # Log the results of interest
    results["availability - time based"].append(availability)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["total annual costs"].append((total + techs) / mil / years)
    results["technicians"].append(techs / mil / years)
    results["materials"].append(parts / mil / years)
    results["vessels"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["dsv cost"].append(dsv / mil / years)
    results["cab cost"].append(cab / mil / years)
    results["manual reset"].append(times.loc[times.index.intersection(["manual reset"]), "downtime"].sum())
    results["minor repair"].append(times.loc[times.index.intersection(["minor repair", ]), "downtime"].sum())
    results["major repair"].append(times.loc[times.index.intersection(["major repair"]), "downtime"].sum())
    results["major replacement"].append(times.loc[times.index.intersection(["major replacement"]), "downtime"].sum())
    results["remote reset"].append(times.loc[times.index.intersection(["remote reset"]), "downtime"].sum())
    results["annual service"].append(times.loc[times.index.intersection(["annual service"]), "downtime"].sum())
    ix = [
        "substructure inspection", "substation inspection",
        "small foundation/scour repair", "cable replacement",
        "small transformer repair", "large transformer repair"
    ]
    results["bos"].append(times.loc[times.index.intersection(ix), "downtime"].sum())
    results["total downtime"].append(times.loc[:, "downtime"].sum())
    results["ctv utilization"].append(ctv_ur)
    results["hlv utilization"].append(hlv_ur)
    results["dsv utilization"].append(dsv_ur)
    results["cab utilization"].append(cab_ur)

In [6]:
# Save the results
# pickled dictionary format
with open(IEA_26/ "results" / "results_dict_v0.6.0.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df.to_csv(IEA_26 / "results" / "results_data_v0.6.0.csv", index_label="result")

In [7]:
results_df

,requests,requests_100pct_reduction,one_mobilization,one_mobilization_100pct_reduction,two_mobilizations,two_mobilizations_100pct_reduction,three_mobilizations,three_mobilizations_100pct_reduction
availability - time based,0.92,0.00,0.79,0.00,0.88,0.01,0.92,0.00
availability - production based,0.93,0.47,0.79,0.29,0.89,0.45,0.93,0.40
capacity factor - net,0.55,0.28,0.46,0.17,0.52,0.26,0.55,0.23
capacity factor - gross,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59
power production,"38,549.95","19,448.12","32,576.33","11,765.54","36,553.78","18,378.21","38,353.71","16,353.45"
task completion rate,0.99,1.00,1.00,1.00,1.00,1.00,0.99,1.00
total annual costs,13.82,14.79,13.71,13.63,16.67,16.56,20.17,20.02
technicians,3.00,3.00,3.00,3.00,3.00,3.00,3.00,3.00
materials,3.20,3.13,3.10,2.98,3.08,3.07,3.25,3.12
vessels,7.62,8.65,7.61,7.64,10.58,10.49,13.92,13.90
